In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
import random
from heapq import heappop, heappush

In [2]:
from cbs import CBS
from Improvements import BP_CBS, PC_CBS

In [3]:
from MAPF import Map
from MAPF import read_map_from_movingai_file, read_tasks_from_movingai_file

from CT import HighNode, LowNode

from OpenClosed import OpenHigh, OpenLow, ClosedLow

from Heuristics import ManhattanDistance

from CT import MakePath
from AstarTimeSteps import AstarTimesteps

In [4]:
from ma_cbs import MetaAgent, MAConflict, CheckMerge, Merge,  MACBS

In [5]:
"""-------Вывод должен быть такой: (PATH_FOUND(BOOL), GOAL_NODES(ITERABLE), nodesGenerated, nodesExpanded)-------"""
import random
 
def massive_test(search_function, max_num, min_num, step, border, isMR, map_file_names, task_file_folders):
    stat = dict()
    taskMap = Map()
 
    stat["corr"] = []
    stat["len"] = []
    stat["generated"] = []
    stat["expanded"] = []
 
    for num in range(len(map_file_names)):
        nodes_created = 0
        num_steps = 0
        correct_true = 0
        correct_false = 0
        error_factor = 0
 
        width, height, cells = read_map_from_movingai_file(map_file_names[num])
 
        taskMap = Map()
        taskMap.SetGridCells(width, height, cells)
 
        for num_agents in range(min_num, max_num + 1, step):
            #считываем файлы из нужной папки (в каждом файле ОДНА начальная конфигурация)
            taskfiles = [f for f in listdir(task_file_folders[num])]       
 
            max_taskCount = 2
 
            for taskCount in [random.randint(0, len(taskfiles)) for i in range(max_taskCount)]:
                #теперь тут уже 2Д массив для К агентов
                task = read_tasks_from_movingai_file(taskfiles[taskCount])
                iStarts = task[:num_agents, 0].astype('int')
                jStarts = task[:num_agents, 1].astype('int')
                iGoals = task[:num_agents, 2].astype('int')
                jGoals = task[:num_agents, 3].astype('int')
 
                tot_length = task[:num_agents, 4].sum()
 
                Starts = [[iStarts[i], jStarts[i]] for i in len(iStarts)]
                Goals = [[iGoals[i], jGoals[i]] for i in len(iGoals)]
        
                if search_function == MACBS:
                    result = MACBS(gridMap=taskMap, Starts=Starts, Goals=Goals, Border=border, isMR=isMR)
                else:
                    result = search_function(gridMap=taskMap, Starts=Starts, Goals=Goals)

                gen = result[2]
                exp = result[3]
 
                # нашли решение
                if result[0]:
                    stat["len"].append(result[1].g)
                    correct = int(paths[1])  == int(tot_length)
 
                    correct_true += correct
                    correct_false += not correct
                    nodes_generated += gen
                    nodes_expanded += exp
                    #error_factor += (int(result[1].g) / int(length) - 1)*100
 
                    stat["corr"].append(correct)
    
                    ################## рисование картинок ##################
 
                    obs_x = []
                    obs_y = []
 
                    for y, cell_y in enumerate(task_map.cells):
                        for x, cell_x in enumerate(cell_y):
                            if cell_x == 1:
                                obs_x.append(x)
                                obs_y.append(y)
 
                    x = np.array(obs_x)
                    y = np.array(obs_y)
 
                    fig = plt.figure(figsize=(5, 5))
                    ax = fig.add_subplot(111)
                    ax.set_ylim(task_map.height, 0)
                    ax.xaxis.set_ticks_position('top')
 
                    ax.scatter(x, y, c='black', marker='s', s=50)
                    ax.set_facecolor('white')
                    ax.set_title(map_file)
 
                    color = ["#" + ''.join([random.choice('0123456789ABCDEF') for j in range(6)]) + ''.join(['A', '0'])
                             for i in range(num)]
 
                    for ma in result[1].sol:
                        for a in result[1].sol[ma]:
                            path = result[1].sol[ma][a]
                            x = []
                            y = []
                            for node in path:
                                x.append(node.i)
                                y.append(node.j)
                            ax.scatter(x[1:-1], y[1:-1], c=color[a], marker='s', s=50)
                            ax.scatter(x[:1], y[:1], c='blue', marker='s', s=50)
                            ax.scatter(x[-1:], y[-1:], c='red', marker='s', s=50)
 
                    plt.show()
                    plt.savefig("images/img" + str(num_agents) + " " + map_file_names[num] + " " + str(taskCount) + ".png")
 
                else:
                    print("Path not found!")
                    stat["corr"].append(False)
                    stat["len"].append(0.0)
 
                stat["generated"].append(gen)
                stat["expanded"].append(exp)

In [7]:
# граница для MA-CBS
border = 3
 
# максимальное число агентов
max_num = 10
 
# минимальное число агентов
min_num = 4
 
# шаг для перебора числа агентов
step = 2

# список карт
map_file_names = ['tests/maze-32-32-4.map', "tests/room-32-32-4.map", "warehouse-10-20-10-2-1.map"]

# список тасок к картам
task_file_folders = ['tests/tests-maze', "tests/scen-even", "tests/scen-even-3"]

for func in [CBS, MACBS, PC_CBS, BP_CBS]: #CBSDS
    if func == MACBS:
        # без MR
        massive_test(func, max_num, min_num, step, border, False, map_file_names, task_file_folders)
        # c MR
        massive_test(func, max_num, min_num, step, border, True, map_file_names, task_file_folders)
    else:
        massive_test(func, max_num, min_num, step, border, False, map_file_names, task_file_folders)

NameError: name 'listdir' is not defined